## Dependencies and dataset import

In [ ]:
!pip install opencv-contrib-python==4.4.0.44
!pip install opencv-contrib-python

In [ ]:
import numpy as np
import random
import os
import cv2 as cv
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.cluster import KMeans
import ipywidgets as widgets
from ipywidgets import interact
from google.colab.patches import cv2_imshow

seed = 123;

Locate in the Google Drive folder corresponding to the dataset

In [ ]:
%ls

 ClickMe.html  'MSR Cambridge EULA for Digital Images_download.rtf'
 GT/            test_set.txt
 IMAGES/


In [ ]:
%cd drive/MyDrive/ISPR MIDTERMS/2-A2/

[Errno 2] No such file or directory: 'drive/MyDrive/ISPR MIDTERMS/2-A2/'
/content/drive/MyDrive/ISPR MIDTERMS/2-A2/DATASET


In [ ]:
%cd DATASET

[Errno 2] No such file or directory: 'DATASET'
/content/drive/MyDrive/ISPR MIDTERMS/2-A2/DATASET


In [ ]:
images = []

# Images are strictly alphabetically sorted, (1_11 comes before 1_9). 
# What we actually care is that they belong to the same class at blocks of 30s. 
# First 30 images belong to class 0, next 30 belong to class 1 and so on....

for filename in sorted(os.listdir('IMAGES')):
    images.append(cv.imread(os.path.join('IMAGES', filename)))

## Train Test Split

Define split percentages, and randomply separate train and test sets picking indexes. 

In [ ]:
CLASS_CARDINALITY = 30
CLASS_NUMBER = 8

In [ ]:
TEST_INDEX_OFFSET = 0 #in range(0,30)

In [ ]:
# Pick all images at the same index for each class to create the test set
test_ids = [TEST_INDEX_OFFSET + CLASS_CARDINALITY * i for i in range(CLASS_NUMBER) ]
training_ids = [ i for i in range(len(images))  if i not in test_ids ] 

training_images = [images[index] for index in training_ids]
test_images = [images[index] for index in test_ids]

print(len(training_images), test_ids)

232 [0, 30, 60, 90, 120, 150, 180, 210]


## Compute the SIFT descriptors for the training set



From every image we first detect most important reason according to MSER. 

For that regions we then comput the SIFT descriptors, obtaining a list of variable lenght as the feature map of each image. 

In [ ]:
def imageToDescriptors(image, save_image=False, all_image_sift=False): 
  descriptors = []
  gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)  

  # Use MSER to find ROIs within image
  mser = cv.MSER_create()
  regions, boxes = mser.detectRegions(image)

  # Compute Sift descriptors for each region detected by MSER.
  sift = cv.xfeatures2d.SIFT_create()
  
  to_save_boxes = []
  to_save_regions = []
  to_save_kp= []

  if(all_image_sift):
      kp, ds = sift.detectAndCompute(roi, None)
      print(len(kp, ds))

  else: 
    for i in range(len(boxes)):
      box = boxes[i]
      x, y, w, h = box;      
      roi = gray[y:y+h,x:x+w]
    
      kp, ds = sift.detectAndCompute(roi, None)

      if np.any(ds):
        descriptors += list(ds)
        if save_image:
          to_save_boxes.append(box)
          to_save_regions.append(regions[i])
          to_save_kp.append(kp)

    
  return (np.array(descriptors, dtype='d'), to_save_boxes, to_save_regions, to_save_kp )

To convert descriptors to feature maps we should first define a codebook.

Let's pcik some images from each class, sample descriptors and run KMeans on them, obtaining a verosimilar representation of our visual descriptors "lexicon".

In [ ]:
SAMPLES_PER_CLASS = 4

In [ ]:
%%time
lexicon_sample = np.empty((0,128))

for i in range(CLASS_NUMBER):
  #1 has been used given that first elements belnogs to test set.
  for j in range(1, SAMPLES_PER_CLASS):
    #Pick first 5 element of each class for the sampling
    descriptors =  imageToDescriptors(images[i*CLASS_CARDINALITY+j], save_image=False)[0]
    lexicon_sample = np.vstack((lexicon_sample, descriptors))
lexicon_sample = np.array(lexicon_sample)

CPU times: user 4.43 s, sys: 1.14 s, total: 5.57 s
Wall time: 4.44 s


In [ ]:
lexicon_sample.shape

(15428, 128)

## Run KMeans on the list of training words found



Divide the space of the SIFT visual descriptors (each consisting in 128 elements array), in 500 clusters, each corresponding to a "word". 

In [ ]:
CLUSTER_NUMBER = 500

In [ ]:
%%time
clustering = KMeans(n_clusters=CLUSTER_NUMBER, random_state=0).fit(lexicon_sample)

CPU times: user 1min 27s, sys: 22 s, total: 1min 49s
Wall time: 1min 5s


Now we can use the predict function of the clustering object to map descriptor into "words".

## Map Images within the dataset to BOW array

To convert images to BOW compute descriptors, classify them as "words" (finding the nearest cluster), and count occurrence of each of the defined words (i.e. 500). 
Every image will correspond to an Histogram. 

In [ ]:
def imageToBOW(image): 
  BOW = np.zeros(CLUSTER_NUMBER)
  descriptors = imageToDescriptors(image)[0]

  predicted_labels = clustering.predict(X = descriptors)
  for p in predicted_labels:  
    BOW[p] += 1
  
  return BOW

In [ ]:
train_bows = []

for image in training_images: 
  train_bows.append(imageToBOW(image))

train_bows = np.array(train_bows)

## Train the LDA Model

Once histograms have been computed we can train the LDA. 

Several Hyperparameters should be considered for a deepen model selection. 
Numper of topics is imposed equal to the number of classes in the dataset, so that we can consider their distribution uniform.

In [ ]:
TOPIC_NUMBER = 8

In [ ]:
lda = LatentDirichletAllocation(n_components=TOPIC_NUMBER, random_state=0, verbose=True, doc_topic_prior=None)
lda.fit(train_bows)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


LatentDirichletAllocation(n_components=8, random_state=0, verbose=True)

## Inference on Test Set

In [ ]:
row = lda.components_[:,0]
print(row/sum(row))

[0.09024824 0.10895755 0.28380066 0.29248705 0.00312875 0.20480415
 0.00312777 0.01344584]


To do inference and evaluate test image we first should convert them to BOW and the feed them to the LDA, obtaining a vector of probabilities (summing to 1). 

In [ ]:
test_data = []

In [ ]:
lda.get_params()

{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 8,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': 0,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': True}

LDA provide us a probability distribution for each topci associated to each word. Using such value we can analyze impact of the single descriptors, plotting them on the original image understanding semantic the semantic of the ROIs. 

In [ ]:
topic_colors = sns.hls_palette(TOPIC_NUMBER)

In [ ]:
color_topic_mapping  = []

In [ ]:
for c in topic_colors: 
  r,g,b = c
  color_topic_mapping.append((r*255, g*255, b*255))

In [ ]:
lda.components_.shape

(8, 500)

In [ ]:
def draw_descriptors(image, plot_boxes=True, plot_regions=True, plot_keypoints=False):

  descriptors, boxes, regions, kp = imageToDescriptors(image, save_image=True)
  signed_image = image.copy()
  for i in range(len(boxes)): 
    word = clustering.predict(X = [descriptors[i]])[0]
    topic = np.argmax(lda.components_[:,word])

    topic_color = color_topic_mapping[topic]
    
    x, y, w, h = boxes[i]

    if plot_boxes:
      cv.rectangle(signed_image, (x, y), (x+w, y+h), topic_color, 2)

    if plot_regions: 
      for p in regions[i]:
        cv.circle(signed_image, (int(p[0]),int(p[1])), radius=0, color=topic_color)

    if plot_keypoints: 
      for p in kp[i]: 
        resultimage = cv.circle(signed_image, (int(p.pt[0]+x), int(p.pt[1]+y)), radius=8, color=topic_color, thickness=1)
        resultimage = cv.circle(signed_image, (int(p.pt[0]+x), int(p.pt[1]+y)), radius=4, color=topic_color, thickness=1)

  cv2_imshow(signed_image)

In [ ]:
%%time
test_topic_dist = [lda.transform( imageToBOW(image).reshape(1, -1))[0] for image in test_images]

CPU times: user 1.35 s, sys: 353 ms, total: 1.71 s
Wall time: 1.08 s


In [61]:
show_topic = []

show_region = widgets.Checkbox(
    value=False,
    description='Show MSERs:',
    disabled=False,
)

show_boxes = widgets.Checkbox(
    value=False,
    description='Show Boxes:',
    disabled=False,
)

show_points = widgets.Checkbox(
    value=False,
    description='Show Key Points:',
    disabled=False,
)

idx = widgets.IntSlider(
    value=0,
    min=0,
    max=7,
    step=1,
    description='Test element index:',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
)

@interact(i=idx, r=show_region, b=show_boxes, p=show_points)
def fun(i, r, b, p):
    
    dist = test_topic_dist[i]
    image = test_images[i]
    main_topic = np.argmax(dist)

    draw_descriptors(image, plot_regions=r, plot_boxes=b, plot_keypoints=p)
    print("Topic probabilities for the image:" )
    print(dist.round(decimals=4))
    print("Main topic: ------->", main_topic )

interactive(children=(IntSlider(value=0, description='Test element index:', max=7), Checkbox(value=False, desc…

Topic Color Mapping

In [ ]:
topic_colors

[(0.86, 0.3712, 0.33999999999999997),
 (0.86, 0.7612000000000001, 0.33999999999999997),
 (0.5688000000000001, 0.86, 0.33999999999999997),
 (0.33999999999999997, 0.86, 0.5012000000000001),
 (0.33999999999999997, 0.8287999999999999, 0.86),
 (0.33999999999999997, 0.43879999999999986, 0.86),
 (0.6311999999999998, 0.33999999999999997, 0.86),
 (0.86, 0.33999999999999997, 0.6987999999999996)]